In [1]:
import clang.cindex
from collections import namedtuple
from typing import *
import re

## Utils

In [2]:
class src_list:          
    def __init__(self):
        self.cur_indent = 0;
        self.src = ""
        
    def __iadd__(self, l:str):
        self.src += " "*self.cur_indent + l + "\n";
        return self
    
    def __rshift__(self, ind):   self.cur_indent += ind;
    def __lshift__(self, ind): self.cur_indent -= ind;

    class indent_ctx:
        def __init__(self, slist, ind:int):
            self.src_list = slist
            self.indent = ind
        def __enter__(self):
            self.src_list >> self.indent
        def __exit__(self, *args):
            self.src_list << self.indent
        
    def indent(self, ind:int=4):
        return src_list.indent_ctx(self, ind)
    
    def get(self):
        return self.src

In [3]:
sl = src_list()
sl += "class A {"
with sl.indent(2):
    sl += "public:"
with sl.indent():
    sl += "int a() { return pmember_a; }"
with sl.indent(2):
    sl += "private:"
    with sl.indent(2):
        sl += "int pmember_a;"
sl += "}"
print(sl.get())

class A {
  public:
    int a() { return pmember_a; }
  private:
    int pmember_a;
}



In [4]:
# export
def enumerate_nodes(node, node_kind:clang.cindex.CursorKind):
    def find_nodes(node, node_kind:clang.cindex.CursorKind):
        nodes = []
        if node.kind == node_kind:
            nodes.append(node)
        for c in node.get_children():
            nodes += find_nodes(c, node_kind)
        return nodes
    
    nodes = find_nodes(node, node_kind)
    node_map = {node.spelling: node for node in nodes}
    return node_map

In [5]:
# export
def split_uppercase(s):
    return [a for a in re.split(r'([A-Z][a-z]*)', s) if a]

def structure_type_name(type_name:str):
    parts = split_uppercase(type_name)[1:]
    stype_parts = ['vk','structure','type']+parts
    return "_".join([s.upper() for s in stype_parts])

In [6]:
def camel_to_snake(s:str, remove_prefixes=False):
    parts = split_uppercase(s)
    if remove_prefixes:
        while parts[0] in ['p', 'pp']:
            parts = parts[1:]
    return "_".join([s.lower() for s in parts])

In [7]:
camel_to_snake("queueFamilyIndexCount"), camel_to_snake("pNext", True)

('queue_family_index_count', 'next')

In [8]:
# export
member = namedtuple("Member", ["type", "name"])
def list_members(node)->List[member]:
    members = []
    for n in node.get_children():
        members.append(member(n.type.spelling, n.spelling))
    return members

### Map between cory and Vulkan types automatically
For example, in the automatically generated interfaces we want to use `glm::uvec3` instead of `VkExtent3D`. To define the mapping between the two types, we define bidirectional mappers that encapsulate the code fragment to insert in order to map a variable of one type to the other.

In [9]:
class type_mapper:
    def __init__(self, vk_type, cory_type, v2c, c2v):
        self.vk_type = vk_type
        self.cory_type = cory_type
        self.c2v = c2v
        self.v2c = v2c
        
    def cory_to_vk(self, var_name:str):
        return self.c2v.format_map({'var':var_name})
    def vk_to_cory(self, var_name:str):
        return self.v2c.format_map({'var':var_name})

In [10]:
cory_type_mappers = {}
vulkan_type_mappers = {}
def register_type_mapper(tm:type_mapper):
    cory_type_mappers[tm.cory_type] = tm
    vulkan_type_mappers[tm.vk_type] = tm
def find_vk_mapper(vk_type):
    if vk_type in vulkan_type_mappers:
        return vulkan_type_mappers[vk_type]
    return type_mapper(vk_type, vk_type, "{var}", "{var}")
def find_cory_mapper(cory_type):
    if cory_type in cory_type_mappers:
        return cory_type_mappers[cory_type]
    return type_mapper(cory_type, cory_type, "{var}", "{var}")

In [11]:
register_type_mapper(
    type_mapper("VkExtent3D", "glm::uvec3", 
                 "glm::uvec3{{ {var}.width, {var}.height, {var}.depth }}",
                 "VkExtent3D{{ {var}.x, {var}.y, {var}.z }}"
                ))

In [67]:
register_type_mapper(
    type_mapper("const char *const *", "std::vector<const char*>", 
                 "undefined",
                 "{var}.data()"
                ))

## Parsing vulkan.h

In [12]:
index = clang.cindex.Index.create()

In [13]:
VULKAN_HEADER_FILE = R"C:\Users\wj89\.conan\data\vulkan-headers\1.2.154.0\_\_\package\5ab84d6acfe1f23c4fae0ab88f26e3a396351ac9\include\vulkan\vulkan_core.h"
#VULKAN_HEADER_FILE = R"ThirdParty/include/flextVk.h"

In [14]:
translation_unit = index.parse(VULKAN_HEADER_FILE)

In [15]:
translation_unit

In [16]:
vk_structs = enumerate_nodes(translation_unit.cursor, clang.cindex.CursorKind.STRUCT_DECL)
vk_enums = enumerate_nodes(translation_unit.cursor, clang.cindex.CursorKind.ENUM_DECL)
vk_funcs = enumerate_nodes(translation_unit.cursor, clang.cindex.CursorKind.FUNCTION_DECL)

In [17]:
vk_create_info_names = [k for k in vk_structs.keys() if k.find("CreateInfo") >=0]

In [18]:
ici_node = vk_structs["VkImageCreateInfo"]

In [19]:
list_members(ici_node)

[Member(type='VkStructureType', name='sType'),
 Member(type='const void *', name='pNext'),
 Member(type='VkImageCreateFlags', name='flags'),
 Member(type='VkImageType', name='imageType'),
 Member(type='VkFormat', name='format'),
 Member(type='VkExtent3D', name='extent'),
 Member(type='uint32_t', name='mipLevels'),
 Member(type='uint32_t', name='arrayLayers'),
 Member(type='VkSampleCountFlagBits', name='samples'),
 Member(type='VkImageTiling', name='tiling'),
 Member(type='VkImageUsageFlags', name='usage'),
 Member(type='VkSharingMode', name='sharingMode'),
 Member(type='uint32_t', name='queueFamilyIndexCount'),
 Member(type='const uint32_t *', name='pQueueFamilyIndices'),
 Member(type='VkImageLayout', name='initialLayout')]

## Creating a builder for the `*CreateInfo` structs

In [20]:
member_defaults = {
    member('VkImageFormat', 'format'): "VK_IMAGE_FORMAT_UNDEFINED",
    member('VkExtent2D', 'extent'): "{1, 1}",
    member('VkExtent3D', 'extent'): "{1, 1, 1}",
    member('uint32_t', 'mipLevels'): "1",
    member('uint32_t', 'arrayLayers'): "1",
    member('VkSampleCountFlagBits', 'samples'): "VK_SAMPLE_COUNT_1_BIT",
}

In [21]:
class builder_builder:
    def __init__(self, built_class, node, with_usage = False):
        self.name = f"{built_class}_builder"
        self.built_class = built_class
        self.node = node
        self.struct_name = self.node.spelling
        self.struct_members = list_members(self.node)
        self.with_usage = with_usage
    
    def process_members(self, src):
        for member in self.struct_members:
            if member.name == 'sType': continue
            if member.name == 'usage' and self.with_usage: continue
            setter_name = camel_to_snake(member.name, remove_prefixes=True)
            mapper = find_vk_mapper(member.type)
            src += f"[[nodiscard]] {self.name}& {setter_name}({mapper.cory_type} {member.name}) noexcept {{"
            with src.indent():
                src += f"info_.{member.name} = {mapper.cory_to_vk(member.name)};"
                src += "return *this;"
            src += "}\n"          
    
    def usage_setter(self, src):
        if not self.with_usage: return
        src += f"[[nodiscard]] {self.name}& usage(device_memory_usage usage) {{"
        with src.indent():
            src += "usage_ = usage;"
            src += "return *this;"
        src += "}"
        
    def name_setter(self, src):
        if not self.with_usage: return
        src += f"[[nodiscard]] {self.name}& name(std::string_view name) {{"
        with src.indent():
            src += "name_ = name;"
            src += "return *this;"
        src += "}"
    
    def constructor(self, src):
        src += f"{self.name}(graphics_context& context): ctx_{{context}} {{}}\n"

    def conversion_operator(self, src):
        src += f"[[nodiscard]] operator {self.built_class}() {{"
        with src.indent():
            src += f"return ctx_.create_{self.built_class}(*this);"
        src += "}\n"
        
    def create_function(self, src):
        src += f"[[nodiscard]] {self.built_class} create() {{";
        with src.indent():
            src += f"return ctx_.create_{self.built_class}(*this);"
        src += "}"
    
    def default_values(self, src):
        src += f".sType = {structure_type_name(self.struct_name)},"
        for m in self.struct_members:
            if m in member_defaults:
                src += f".{m.name} = {member_defaults[m]},"
        
    def build(self):
        src = src_list()
        src += f"class {self.name} {{"
        with src.indent(2):
            src += "public:"
            
        with src.indent():
            src += "friend class graphics_context;"
            self.constructor(src)
            self.process_members(src)
            self.usage_setter(src)
            self.name_setter(src)
            self.conversion_operator(src)
            self.create_function(src)
        
        with src.indent(2):
            src += "private:"
        with src.indent(4):
            src += "graphics_context& ctx_;"
            src += f"{self.struct_name} info_{{"
            with src.indent():
                self.default_values(src)
            src += "};"
            src += "std::string_view name_;"
            if self.with_usage:
                src += "device_memory_usage usage_{VMA_MEMORY_USAGE_GPU_ONLY};"
        src += "};"
        
        return src.src

In [126]:
b = builder_builder("image", vk_structs["VkImageCreateInfo"], with_usage=True)
print(b.build())

class image_builder {
  public:
    friend class graphics_context;
    image_builder(graphics_context& context): ctx_{context} {}

    [[nodiscard]] image_builder& next(const void * pNext) noexcept {
        info_.pNext = pNext;
        return *this;
    }

    [[nodiscard]] image_builder& flags(VkImageCreateFlags flags) noexcept {
        info_.flags = flags;
        return *this;
    }

    [[nodiscard]] image_builder& image_type(VkImageType imageType) noexcept {
        info_.imageType = imageType;
        return *this;
    }

    [[nodiscard]] image_builder& format(VkFormat format) noexcept {
        info_.format = format;
        return *this;
    }

    [[nodiscard]] image_builder& extent(glm::uvec3 extent) noexcept {
        info_.extent = VkExtent3D{ extent.x, extent.y, extent.z };
        return *this;
    }

    [[nodiscard]] image_builder& mip_levels(uint32_t mipLevels) noexcept {
        info_.mipLevels = mipLevels;
        return *this;
    }

    [[nodiscard]] image_build

In [127]:
b = builder_builder("image_view", vk_structs["VkImageViewCreateInfo"])
print(b.build())

class image_view_builder {
  public:
    friend class graphics_context;
    image_view_builder(graphics_context& context): ctx_{context} {}

    [[nodiscard]] image_view_builder& next(const void * pNext) noexcept {
        info_.pNext = pNext;
        return *this;
    }

    [[nodiscard]] image_view_builder& flags(VkImageViewCreateFlags flags) noexcept {
        info_.flags = flags;
        return *this;
    }

    [[nodiscard]] image_view_builder& image(VkImage image) noexcept {
        info_.image = image;
        return *this;
    }

    [[nodiscard]] image_view_builder& view_type(VkImageViewType viewType) noexcept {
        info_.viewType = viewType;
        return *this;
    }

    [[nodiscard]] image_view_builder& format(VkFormat format) noexcept {
        info_.format = format;
        return *this;
    }

    [[nodiscard]] image_view_builder& components(VkComponentMapping components) noexcept {
        info_.components = components;
        return *this;
    }

    [[nodiscard

In [128]:
b = builder_builder("buffer", vk_structs["VkBufferCreateInfo"], with_usage=True)
print(b.build())

class buffer_builder {
  public:
    friend class graphics_context;
    buffer_builder(graphics_context& context): ctx_{context} {}

    [[nodiscard]] buffer_builder& next(const void * pNext) noexcept {
        info_.pNext = pNext;
        return *this;
    }

    [[nodiscard]] buffer_builder& flags(VkBufferCreateFlags flags) noexcept {
        info_.flags = flags;
        return *this;
    }

    [[nodiscard]] buffer_builder& size(VkDeviceSize size) noexcept {
        info_.size = size;
        return *this;
    }

    [[nodiscard]] buffer_builder& sharing_mode(VkSharingMode sharingMode) noexcept {
        info_.sharingMode = sharingMode;
        return *this;
    }

    [[nodiscard]] buffer_builder& queue_family_index_count(uint32_t queueFamilyIndexCount) noexcept {
        info_.queueFamilyIndexCount = queueFamilyIndexCount;
        return *this;
    }

    [[nodiscard]] buffer_builder& queue_family_indices(const uint32_t * pQueueFamilyIndices) noexcept {
        info_.pQueueFamilyI

In [129]:
b = builder_builder("instance", vk_structs["VkInstanceCreateInfo"])
print(b.build())

class instance_builder {
  public:
    friend class graphics_context;
    instance_builder(graphics_context& context): ctx_{context} {}

    [[nodiscard]] instance_builder& next(const void * pNext) noexcept {
        info_.pNext = pNext;
        return *this;
    }

    [[nodiscard]] instance_builder& flags(VkInstanceCreateFlags flags) noexcept {
        info_.flags = flags;
        return *this;
    }

    [[nodiscard]] instance_builder& application_info(const VkApplicationInfo * pApplicationInfo) noexcept {
        info_.pApplicationInfo = pApplicationInfo;
        return *this;
    }

    [[nodiscard]] instance_builder& enabled_layer_count(uint32_t enabledLayerCount) noexcept {
        info_.enabledLayerCount = enabledLayerCount;
        return *this;
    }

    [[nodiscard]] instance_builder& enabled_layer_names(const char *const * ppEnabledLayerNames) noexcept {
        info_.ppEnabledLayerNames = ppEnabledLayerNames;
        return *this;
    }

    [[nodiscard]] instance_builder

In [66]:
print(builder_builder("queue", vk_structs["VkDeviceQueueCreateInfo"]).build())
print(builder_builder("device", vk_structs["VkDeviceCreateInfo"]).build())

class queue_builder {
  public:
    friend class graphics_context;
    queue_builder(graphics_context& context): ctx_{context} {}

    [[nodiscard]] queue_builder& next(const void * pNext) noexcept {
        info_.pNext = pNext;
        return *this;
    }

    [[nodiscard]] queue_builder& flags(VkDeviceQueueCreateFlags flags) noexcept {
        info_.flags = flags;
        return *this;
    }

    [[nodiscard]] queue_builder& queue_family_index(uint32_t queueFamilyIndex) noexcept {
        info_.queueFamilyIndex = queueFamilyIndex;
        return *this;
    }

    [[nodiscard]] queue_builder& queue_count(uint32_t queueCount) noexcept {
        info_.queueCount = queueCount;
        return *this;
    }

    [[nodiscard]] queue_builder& queue_priorities(const float * pQueuePriorities) noexcept {
        info_.pQueuePriorities = pQueuePriorities;
        return *this;
    }

    [[nodiscard]] operator queue() {
        return ctx_.create_queue(*this);
    }

    [[nodiscard]] queue creat

In [68]:
print(builder_builder("debug_utils_messenger", vk_structs["VkDebugUtilsMessengerCreateInfoEXT"]).build())

class debug_utils_messenger_builder {
  public:
    friend class graphics_context;
    debug_utils_messenger_builder(graphics_context& context): ctx_{context} {}

    [[nodiscard]] debug_utils_messenger_builder& next(const void * pNext) noexcept {
        info_.pNext = pNext;
        return *this;
    }

    [[nodiscard]] debug_utils_messenger_builder& flags(VkDebugUtilsMessengerCreateFlagsEXT flags) noexcept {
        info_.flags = flags;
        return *this;
    }

    [[nodiscard]] debug_utils_messenger_builder& message_severity(VkDebugUtilsMessageSeverityFlagsEXT messageSeverity) noexcept {
        info_.messageSeverity = messageSeverity;
        return *this;
    }

    [[nodiscard]] debug_utils_messenger_builder& message_type(VkDebugUtilsMessageTypeFlagsEXT messageType) noexcept {
        info_.messageType = messageType;
        return *this;
    }

    [[nodiscard]] debug_utils_messenger_builder& pfn_user_callback(PFN_vkDebugUtilsMessengerCallbackEXT pfnUserCallback) noexcept {

In [93]:
enumerate_nodes(translation_unit.cursor, clang.cindex.CursorKind.FUNCTION_DECL)

{'__va_start': <clang.cindex.Cursor at 0x18234d606c8>,
 '__security_init_cookie': <clang.cindex.Cursor at 0x18234d60a48>,
 '__security_check_cookie': <clang.cindex.Cursor at 0x18234d60ac8>,
 '__report_gsfailure': <clang.cindex.Cursor at 0x18234d60bc8>,
 '_invalid_parameter_noinfo': <clang.cindex.Cursor at 0x18234d60cc8>,
 '_invalid_parameter_noinfo_noreturn': <clang.cindex.Cursor at 0x18234d60d48>,
 '_invoke_watson': <clang.cindex.Cursor at 0x18234d60dc8>,
 '_errno': <clang.cindex.Cursor at 0x18234d645c8>,
 '_set_errno': <clang.cindex.Cursor at 0x18234d64648>,
 '_get_errno': <clang.cindex.Cursor at 0x18234d646c8>,
 '__threadid': <clang.cindex.Cursor at 0x18234d64748>,
 '__threadhandle': <clang.cindex.Cursor at 0x18234d647c8>,
 'vkCreateInstance': <clang.cindex.Cursor at 0x18234d049c8>,
 'vkDestroyInstance': <clang.cindex.Cursor at 0x18234d04a48>,
 'vkEnumeratePhysicalDevices': <clang.cindex.Cursor at 0x18234d04ac8>,
 'vkGetPhysicalDeviceFeatures': <clang.cindex.Cursor at 0x18234d04b48>

## Generate enum to_string serializers

In [59]:
enum_name = namedtuple("EnumName", ["type", "name", "value"])

In [30]:
def list_enum_names(node):
    members = []
    for n in node.get_children():
        members.append(enum_name(n.type.spelling, n.spelling, n.enum_value))
    return members

In [35]:
list_enum_names(vk_enums['VkImageTiling'])

[EnumName(type='int', name='VK_IMAGE_TILING_OPTIMAL', value=0),
 EnumName(type='int', name='VK_IMAGE_TILING_LINEAR', value=1),
 EnumName(type='int', name='VK_IMAGE_TILING_DRM_FORMAT_MODIFIER_EXT', value=1000158000),
 EnumName(type='int', name='VK_IMAGE_TILING_MAX_ENUM', value=2147483647)]

In [32]:
vk_enums

{'VkResult': <clang.cindex.Cursor at 0x18f8d3249c8>,
 'VkStructureType': <clang.cindex.Cursor at 0x18f8d324a48>,
 'VkImageLayout': <clang.cindex.Cursor at 0x18f8d324848>,
 'VkObjectType': <clang.cindex.Cursor at 0x18f8d324ac8>,
 'VkVendorId': <clang.cindex.Cursor at 0x18f8d324948>,
 'VkPipelineCacheHeaderVersion': <clang.cindex.Cursor at 0x18f8d3248c8>,
 'VkSystemAllocationScope': <clang.cindex.Cursor at 0x18f8d324ec8>,
 'VkInternalAllocationType': <clang.cindex.Cursor at 0x18f8d324b48>,
 'VkFormat': <clang.cindex.Cursor at 0x18f8d324cc8>,
 'VkImageTiling': <clang.cindex.Cursor at 0x18f8d324c48>,
 'VkImageType': <clang.cindex.Cursor at 0x18f8d324bc8>,
 'VkPhysicalDeviceType': <clang.cindex.Cursor at 0x18f8d324e48>,
 'VkQueryType': <clang.cindex.Cursor at 0x18f8d324dc8>,
 'VkSharingMode': <clang.cindex.Cursor at 0x18f8d324f48>,
 'VkComponentSwizzle': <clang.cindex.Cursor at 0x18f8d324d48>,
 'VkImageViewType': <clang.cindex.Cursor at 0x18f8d32a048>,
 'VkBlendFactor': <clang.cindex.Cursor

In [51]:
def make_to_string_function(node:clang.cindex.Cursor):
    src = src_list()
    param_name = camel_to_snake(node.spelling);
    src += f"constexpr std::string_view to_string({node.spelling} {param_name}) noexcept {{"
    with src.indent():
        src += f"switch ({param_name}) {{"
        enum_names = list_enum_names(node)
        with src.indent():
            processed_values = []
            for n in enum_names:
                if n.value in processed_values: continue
                if n.name.endswith('_MAX_ENUM'): continue
                processed_values.append(n.value)
                
                src += f"case {n.name}:"
                with src.indent():
                    src += f"return \"{n.name}\";"
        src += "}"
        src += f"return \"Unknown {node.spelling}\";"
    src += "}"
    
    return src.get()

print(make_to_string_function(vk_enums['VkResult']))
print(make_to_string_function(vk_enums['VkImageTiling']))

constexpr std::string_view to_string(VkResult vk_result) noexcept {
    switch (vk_result) {
        case VK_SUCCESS:
            return "VK_SUCCESS";
        case VK_NOT_READY:
            return "VK_NOT_READY";
        case VK_TIMEOUT:
            return "VK_TIMEOUT";
        case VK_EVENT_SET:
            return "VK_EVENT_SET";
        case VK_EVENT_RESET:
            return "VK_EVENT_RESET";
        case VK_INCOMPLETE:
            return "VK_INCOMPLETE";
        case VK_ERROR_OUT_OF_HOST_MEMORY:
            return "VK_ERROR_OUT_OF_HOST_MEMORY";
        case VK_ERROR_OUT_OF_DEVICE_MEMORY:
            return "VK_ERROR_OUT_OF_DEVICE_MEMORY";
        case VK_ERROR_INITIALIZATION_FAILED:
            return "VK_ERROR_INITIALIZATION_FAILED";
        case VK_ERROR_DEVICE_LOST:
            return "VK_ERROR_DEVICE_LOST";
        case VK_ERROR_MEMORY_MAP_FAILED:
            return "VK_ERROR_MEMORY_MAP_FAILED";
        case VK_ERROR_LAYER_NOT_PRESENT:
            return "VK_ERROR_LAYER_NOT_PRE

In [65]:
with open('Cory/include/Cory/vk/enum_utils.h', 'w') as f:
    f.write("""
#pragma once

#include <fmt/format.h>
#include <string_view>
#include <vulkan/vulkan.h>

namespace cory {
namespace vk {

""")
    
    for enum_node in vk_enums.values():
        f.write(make_to_string_function(enum_node))
    
    f.write("""
    
template <typename VulkanFlagBitsType, typename VulkanFlagsType>
std::string flag_bits_to_string(VulkanFlagsType flagBits)
{
        auto curFlag = (VulkanFlagBitsType)1;
    if (!flagBits) return "( )";

    std::string flagBitsString("( ");
    while (curFlag) {
        if (flagBits & curFlag) { flagBitsString += std::string(to_string((VulkanFlagBitsType)curFlag)) + " ";
        }
        curFlag = (VulkanFlagBitsType)(curFlag << 1);
    }
    return flagBitsString + ")";
}
    
} // namespace vk
} // namespace cory

template <> struct fmt::formatter<VkResult> {
    constexpr auto parse(format_parse_context &ctx)
    {
        auto it = ctx.begin(), end = ctx.end();
        if (it != end && *it != '}') throw format_error("invalid format");
        return ctx.end();
    }

    template <typename FormatContext, typename VulkanEnumType>
    auto format(const VulkanEnumType &e, FormatContext &ctx)
    {
        return format_to(ctx.out(), cory::vk::to_string(e));
    }
};

""")
    
    for ename in vk_enums.keys():
        if ename == "VkResult": continue
        f.write(f"template <> struct fmt::formatter<{ename}> : fmt::formatter<VkResult> {{}};\n")
    